In [30]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

pd.set_option('display.max_columns', None)

In [31]:
#carrega o dataset
url = "/datacov.csv"
data = pd.read_csv(url)
#exibe as informações do dataset
data.info()
print(f"Read in {len(data)} rows")

C:\Users\Rafael\AppData\Local\Temp\ipykernel_16932\1080176038.py:3: DtypeWarning: Columns (15,59,61,62,63,91,93,122,143,171) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(url)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554328 entries, 0 to 554327
Columns: 173 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(95), int64(12), object(66)
memory usage: 731.6+ MB
Read in 554328 rows


In [32]:
#data.head()

In [33]:
#Remoção das linhas com 3 e 9
aux = data[(data['EVOLUCAO'] == 3) | (data['EVOLUCAO'] == 9)].index
data.drop(aux , inplace=True)
data.info()
print(f"Read in {len(data)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526276 entries, 0 to 554327
Columns: 173 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(95), int64(12), object(66)
memory usage: 698.6+ MB
Read in 526276 rows


In [34]:
#Realiza a separação dos atributos e rótulos
x = data.drop(columns= 'EVOLUCAO')
y = data['EVOLUCAO']

In [35]:
aux = x[(x['SG_UF_NOT'] != 'PR')].index
x.drop(aux , inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51936 entries, 13 to 554312
Columns: 172 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(94), int64(12), object(66)
memory usage: 68.5+ MB
Read in 51936 rows


In [36]:
#mantém somente diagnóstico de COVID
aux = x[(x['CLASSI_FIN'] != 5)].index
x.drop(aux , inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Columns: 172 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(94), int64(12), object(66)
memory usage: 24.1+ MB
Read in 18222 rows


In [37]:
#x[['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'DOR_ABD',
#   'FADIGA', 'PERD_OLFT', 'PERD_PALA', 'OUTRO_SIN', 'FATOR_RISC', 'PUERPERA', 'CARDIOPATI', 'HEMATOLOGI',
#  'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE',
#  'OUT_MORBI', 'VACINA_COV']].replace(9, np.nan, inplace=True)

In [38]:
x.replace(9, np.nan, inplace=True)

In [39]:
x.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,CO_UNI_NOT,CS_SEXO,DT_NASC,NU_IDADE_N,TP_IDADE,COD_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,ID_PAIS,CO_PAIS,SG_UF,ID_RG_RESI,CO_RG_RESI,ID_MN_RESI,CO_MUN_RES,CS_ZONA,SURTO_SG,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,OUTRO_DES,PUERPERA,FATOR_RISC,CARDIOPATI,HEMATOLOGI,SIND_DOWN,HEPATICA,ASMA,DIABETES,NEUROLOGIC,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OBES_IMC,OUT_MORBI,MORB_DESC,VACINA,DT_UT_DOSE,MAE_VAC,DT_VAC_MAE,M_AMAMENTA,DT_DOSEUNI,DT_1_DOSE,DT_2_DOSE,ANTIVIRAL,TP_ANTIVIR,OUT_ANTIV,DT_ANTIVIR,HOSPITAL,DT_INTERNA,SG_UF_INTE,ID_RG_INTE,CO_RG_INTE,ID_MN_INTE,CO_MU_INTE,UTI,DT_ENTUTI,DT_SAIDUTI,SUPORT_VEN,RAIOX_RES,RAIOX_OUT,DT_RAIOX,AMOSTRA,DT_COLETA,TP_AMOSTRA,OUT_AMOST,PCR_RESUL,DT_PCR,POS_PCRFLU,TP_FLU_PCR,PCR_FLUASU,FLUASU_OUT,PCR_FLUBLI,FLUBLI_OUT,POS_PCROUT,PCR_VSR,PCR_PARA1,PCR_PARA2,PCR_PARA3,PCR_PARA4,PCR_ADENO,PCR_METAP,PCR_BOCA,PCR_RINO,PCR_OUTRO,DS_PCR_OUT,CLASSI_FIN,CLASSI_OUT,CRITERIO,DT_EVOLUCA,DT_ENCERRA,DT_DIGITA,HISTO_VGM,PAIS_VGM,CO_PS_VGM,LO_PS_VGM,DT_VGM,DT_RT_VGM,PCR_SARS2,PAC_COCBO,PAC_DSCBO,OUT_ANIM,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,TOMO_OUT,DT_TOMO,TP_TES_AN,DT_RES_AN,RES_AN,POS_AN_FLU,TP_FLU_AN,POS_AN_OUT,AN_SARS2,AN_VSR,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,DOSE_REF,FAB_COV_1,FAB_COV_2,FAB_COVREF,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV,LOTE_REF,FNT_IN_COV,DOSE_2REF,FAB_COVRF2,LOTE_REF2,TRAT_COV,TIPO_TRAT,OUT_TRAT,DT_TRT_COV
62,25/01/2022,4.0,16/01/2022,3.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL UNIVERSITARIO EVANGELICO MACKENZIE,15245,F,22/08/1969,52.0,3,3052,5.0,NaN,NaN,BRASIL,1,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,NaN,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,"CEFALEIA, MIALGIA",2.0,1,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,"41,9",2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,20/01/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,20/01/2022,09/03/2022,2.0,6.0,NaN,NaN,1.0,21/01/2022,1.0,NaN,1.0,28/01/2022,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,09/03/2022,09/03/2022,25/01/2022,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,6.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,25/05/2021,30/08/2021,NaN,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,NaN,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,213VCD041W,217VCD236W,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,28/01/2022,4.0,28/01/2022,4.0,PR,15RS MARINGA,1369.0,MARINGA,411520,HOSPITAL E MATERNIDADE SAO MARCOS,2586444,M,16/01/1956,66.0,3,3066,6.0,1.0,2.0,BRASIL,1,PR,15RS MARINGA,1369.0,MARINGA,411520.0,1.0,NaN,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,28/01/2022,PR,15RS MARINGA,1369.0,MARINGA,411520.0,2.0,NaN,NaN,3.0,6.0,NaN,NaN,1.0,28/01/2022,1.0,NaN,1.0,28/01/2022,2.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,31/01/2022,17/03/2022,03/02/2022,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,1.0,NaN,28/01/2022,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,19/04/2021,19/07/2021,NaN,FIOCRUZ,FIOCRUZ,NaN,FIOCRUZ,213VCD014W,214VCD103W,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,03/02/2022,5.0,28/01/2022,4.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL ERASTO GAERTNER,15644,M,09/01/1952,70.0,3,3070,6.0,1.0,NaN,BRASIL,1,PR,02RS METROPOLITANA,1356.0,PINHAIS,411915.0,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,CONGESTAO NASAL,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,LLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,02/02/2022,

In [40]:
#Remove colunas irrelevantes
x.drop(['DT_NASC', 'TP_IDADE','CS_ESCOL_N', 'CO_PAIS', 'DT_DIGITA', 'FNT_IN_COV'], axis=1, inplace=True)
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Columns: 166 entries, DT_NOTIFIC to DT_TRT_COV
dtypes: float64(98), int64(4), object(64)
memory usage: 23.2+ MB


In [41]:
#Remove as colunas com todos os dados nulos
x.dropna(axis=1, how='all', thresh=None, subset=None, inplace=True)
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Columns: 141 entries, DT_NOTIFIC to OUT_TRAT
dtypes: float64(80), int64(4), object(57)
memory usage: 19.7+ MB


In [29]:
#remoção das colunas com 50% de dados faltantes
#x.dropna(axis=1, how='any', thresh=9111, subset=None, inplace=True)
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18222 entries, 62 to 554312
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  18222 non-null  object 
 1   DT_SIN_PRI  18222 non-null  object 
 2   SG_UF_NOT   18222 non-null  object 
 3   ID_REGIONA  18222 non-null  object 
 4   CO_REGIONA  18222 non-null  float64
 5   ID_MUNICIP  18222 non-null  object 
 6   CO_MUN_NOT  18222 non-null  int64  
 7   ID_UNIDADE  18222 non-null  object 
 8   CO_UNI_NOT  18222 non-null  int64  
 9   CS_SEXO     18222 non-null  object 
 10  COD_IDADE   18222 non-null  object 
 11  ID_PAIS     18222 non-null  object 
 12  FATOR_RISC  18222 non-null  int64  
 13  CLASSI_FIN  18222 non-null  float64
 14  HISTO_VGM   18222 non-null  int64  
dtypes: float64(2), int64(4), object(9)
memory usage: 2.2+ MB


In [28]:
x.head()

,DT_NOTIFIC,DT_SIN_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,CO_UNI_NOT,CS_SEXO,COD_IDADE,ID_PAIS,FATOR_RISC,CLASSI_FIN,HISTO_VGM
62,25/01/2022,16/01/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL UNIVERSITARIO EVANGELICO MACKENZIE,15245,F,3052,BRASIL,1,5.0,0
80,28/01/2022,28/01/2022,PR,15RS MARINGA,1369.0,MARINGA,411520,HOSPITAL E MATERNIDADE SAO MARCOS,2586444,M,3066,BRASIL,1,5.0,0
99,03/02/2022,28/01/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL ERASTO GAERTNER,15644,M,3070,BRASIL,1,5.0,0
102,07/02/2022,04/02/2022,PR,20RS TOLEDO,1374.0,MARECHAL CANDIDO RONDON,411460,HOSPITAL RONDON,4053907,F,3078,BRASIL,2,5.0,0
175,25/03/2022,16/02/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL INFANTIL PEQUENO PRINCIPE,15563,F,3006,BRASIL,1,5.0,0


In [34]:
#Remoção das linhas com 50% de dados faltantes
x.dropna(axis=0, thresh=38, subset=None, inplace=True)
x.info()
print(f"Read in {len(x)} rows")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18208 entries, 62 to 554312
Data columns (total 77 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DT_NOTIFIC  18208 non-null  object 
 1   SEM_NOT     17830 non-null  float64
 2   DT_SIN_PRI  18208 non-null  object 
 3   SEM_PRI     17921 non-null  float64
 4   SG_UF_NOT   18208 non-null  object 
 5   ID_REGIONA  18208 non-null  object 
 6   CO_REGIONA  18208 non-null  float64
 7   ID_MUNICIP  18208 non-null  object 
 8   CO_MUN_NOT  18208 non-null  int64  
 9   ID_UNIDADE  18208 non-null  object 
 10  CO_UNI_NOT  18208 non-null  int64  
 11  CS_SEXO     18208 non-null  object 
 12  NU_IDADE_N  18123 non-null  float64
 13  COD_IDADE   18208 non-null  object 
 14  CS_GESTANT  17952 non-null  float64
 15  CS_RACA     15389 non-null  float64
 16  ID_PAIS     18208 non-null  object 
 17  SG_UF       18192 non-null  object 
 18  ID_RG_RESI  18184 non-null  object 
 19  CO_RG_RESI  18184 non-n

In [35]:
x.head()

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,CO_UNI_NOT,CS_SEXO,NU_IDADE_N,COD_IDADE,CS_GESTANT,CS_RACA,ID_PAIS,SG_UF,ID_RG_RESI,CO_RG_RESI,ID_MN_RESI,CO_MUN_RES,CS_ZONA,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,SATURACAO,DIARREIA,VOMITO,OUTRO_SIN,FATOR_RISC,CARDIOPATI,DIABETES,OUT_MORBI,ANTIVIRAL,HOSPITAL,DT_INTERNA,SG_UF_INTE,ID_RG_INTE,CO_RG_INTE,ID_MN_INTE,CO_MU_INTE,UTI,SUPORT_VEN,RAIOX_RES,AMOSTRA,DT_COLETA,TP_AMOSTRA,PCR_RESUL,DT_PCR,POS_PCRFLU,POS_PCROUT,CLASSI_FIN,CRITERIO,DT_EVOLUCA,DT_ENCERRA,HISTO_VGM,PCR_SARS2,DOR_ABD,FADIGA,PERD_OLFT,PERD_PALA,TOMO_RES,RES_AN,ESTRANG,VACINA_COV,DOSE_1_COV,DOSE_2_COV,FAB_COV_1,FAB_COV_2,LAB_PR_COV,LOTE_1_COV,LOTE_2_COV
62,25/01/2022,4.0,16/01/2022,3.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL UNIVERSITARIO EVANGELICO MACKENZIE,15245,F,52.0,3052,5.0,NaN,BRASIL,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,2.0,2.0,1.0,20/01/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,2.0,6.0,1.0,21/01/2022,1.0,1.0,28/01/2022,2.0,1.0,5.0,1.0,09/03/2022,09/03/2022,0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,25/05/2021,30/08/2021,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,213VCD041W,217VCD236W
80,28/01/2022,4.0,28/01/2022,4.0,PR,15RS MARINGA,1369.0,MARINGA,411520,HOSPITAL E MATERNIDADE SAO MARCOS,2586444,M,66.0,3066,6.0,1.0,BRASIL,PR,15RS MARINGA,1369.0,MARINGA,411520.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,1.0,2.0,2.0,2.0,1.0,28/01/2022,PR,15RS MARINGA,1369.0,MARINGA,411520.0,2.0,3.0,6.0,1.0,28/01/2022,1.0,1.0,28/01/2022,2.0,1.0,5.0,1.0,31/01/2022,17/03/2022,0,1.0,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,19/04/2021,19/07/2021,FIOCRUZ,FIOCRUZ,FIOCRUZ,213VCD014W,214VCD103W
99,03/02/2022,5.0,28/01/2022,4.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL ERASTO GAERTNER,15644,M,70.0,3070,6.0,1.0,BRASIL,PR,02RS METROPOLITANA,1356.0,PINHAIS,411915.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,1.0,1,NaN,NaN,1.0,2.0,1.0,02/02/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,1.0,6.0,1.0,02/02/2022,1.0,1.0,05/02/2022,2.0,1.0,5.0,1.0,06/02/2022,09/02/2022,0,1.0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,1.0,30/03/2021,23/04/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,210099,210137
102,07/02/2022,6.0,04/02/2022,5.0,PR,20RS TOLEDO,1374.0,MARECHAL CANDIDO RONDON,411460,HOSPITAL RONDON,4053907,F,78.0,3078,5.0,1.0,BRASIL,PR,20RS TOLEDO,1374.0,MARECHAL CANDIDO RONDON,411460.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,07/02/2022,1.0,1.0,09/02/2022,2.0,1.0,5.0,1.0,NaN,NaN,0,1.0,NaN,1.0,NaN,NaN,NaN,5.0,NaN,1.0,08/03/2021,05/04/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,210011A,210056
175,25/03/2022,12.0,16/02/2022,7.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL INFANTIL PEQUENO PRINCIPE,15563,F,6.0,3006,6.0,NaN,BRASIL,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1,2.0,2.0,2.0,2.0,1.0,18/02/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,1.0,5.0,1.0,18/02/2022,1.0,1.0,18/02/2022,2.0,1.0,5.0,1.0,18/03/2022,26/04/2022,0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,21/06/2022,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,NaN,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,B202107083,NaN


In [17]:
pd.set_option('display.max_rows', None)
x.isnull().sum()

Series([], dtype: float64)

In [37]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(x)
  
x = imputer.transform(x)

In [43]:
data = pd.DataFrame(x);
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76
0,25/01/2022,4.0,16/01/2022,3.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL UNIVERSITARIO EVANGELICO MACKENZIE,15245,F,52.0,3052,5.0,1.0,BRASIL,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0,1.0,2.0,2.0,1.0,20/01/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,2.0,6.0,1.0,21/01/2022,1.0,1.0,28/01/2022,2.0,1.0,5.0,1.0,09/03/2022,09/03/2022,0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,25/05/2021,30/08/2021,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,85 - COVID-19 ASTRAZENECA/FIOCRUZ - COVISHIELD,213VCD041W,217VCD236W
1,28/01/2022,4.0,28/01/2022,4.0,PR,15RS MARINGA,1369.0,MARINGA,411520,HOSPITAL E MATERNIDADE SAO MARCOS,2586444,M,66.0,3066,6.0,1.0,BRASIL,PR,15RS MARINGA,1369.0,MARINGA,411520.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1,1.0,2.0,2.0,2.0,1.0,28/01/2022,PR,15RS MARINGA,1369.0,MARINGA,411520.0,2.0,3.0,6.0,1.0,28/01/2022,1.0,1.0,28/01/2022,2.0,1.0,5.0,1.0,31/01/2022,17/03/2022,0,1.0,2.0,2.0,2.0,2.0,1.0,5.0,2.0,1.0,19/04/2021,19/07/2021,FIOCRUZ,FIOCRUZ,FIOCRUZ,213VCD014W,214VCD103W
2,03/02/2022,5.0,28/01/2022,4.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL ERASTO GAERTNER,15644,M,70.0,3070,6.0,1.0,BRASIL,PR,02RS METROPOLITANA,1356.0,PINHAIS,411915.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1,1.0,2.0,1.0,2.0,1.0,02/02/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,1.0,6.0,1.0,02/02/2022,1.0,1.0,05/02/2022,2.0,1.0,5.0,1.0,06/02/2022,09/02/2022,0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,30/03/2021,23/04/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,210099,210137
3,07/02/2022,6.0,04/02/2022,5.0,PR,20RS TOLEDO,1374.0,MARECHAL CANDIDO RONDON,411460,HOSPITAL RONDON,4053907,F,78.0,3078,5.0,1.0,BRASIL,PR,20RS TOLEDO,1374.0,MARECHAL CANDIDO RONDON,411460.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2,1.0,2.0,1.0,2.0,1.0,25/01/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,2.0,2.0,6.0,1.0,07/02/2022,1.0,1.0,09/02/2022,2.0,1.0,5.0,1.0,03/02/2022,16/02/2022,0,1.0,2.0,1.0,2.0,2.0,6.0,5.0,2.0,1.0,08/03/2021,05/04/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,210011A,210056
4,25/03/2022,12.0,16/02/2022,7.0,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,HOSPITAL INFANTIL PEQUENO PRINCIPE,15563,F,6.0,3006,6.0,1.0,BRASIL,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1,2.0,2.0,2.0,2.0,1.0,18/02/2022,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690.0,1.0,1.0,5.0,1.0,18/02/2022,1.0,1.0,18/02/2022,2.0,1.0,5.0,1.0,18/03/2022,26/04/2022,0,1.0,2.0,2.0,2.0,2.0,6.0,4.0,2.0,1.0,21/06/2022,20/04/2021,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,86 - COVID-19 SINOVAC/BUTANTAN - CORONAVAC,B202107083,210130


In [ ]:
from sklearn.model_selection import cross_val_score
for name, clf in zip(names, classifiers):

    score = cross_val_score(clf, x_rfe, y, cv=10)
    print(f"Accuracy of {name} Classifier is:{score}")
    print('Média: {:.2} | Desvio: {:.2}'.format(np.mean(score), np.std(score)))